In [0]:
# LDA Topic Modelling
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/


In [1]:
# Run in python console
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Administrator/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [6]:
import os,sys
import json

In [7]:
# Import Dataset
#df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
#print(df.target_names.unique())
#df.head()
train_df = pd.read_csv('/Users/Administrator/Downloads/FakeNewsWSDM/train.csv')
test_df = pd.read_csv('/Users/Administrator/Downloads/FakeNewsWSDM/test.csv') 

In [0]:
train_df.head()

,id,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label
0,0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated
1,3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated
2,1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated
3,2,2,5,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP topped Hong Kong last year? She...,unrelated
4,9,6,7,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,"""How to discriminate oil from gutter oil by me...",It took 30 years of cooking oil to know that o...,agreed


In [11]:
data1 = train_df.title1_en.values.tolist()
data2 = train_df.title2_en.values.tolist()
data = data1 + data2

In [19]:
data[:1]

['There are two new old-age insurance benefits for old people in rural areas. Have you got them?']

In [18]:
# Remove quotes, newline characters
# Convert to list
#data = train_df.title1_en.values.tolist()

# Remove Emails
#data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['There are two new old-age insurance benefits for old people in rural areas. '
 'Have you got them?']


In [0]:
# Tokenize words and Clean-up text

In [20]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['there', 'are', 'two', 'new', 'old', 'age', 'insurance', 'benefits', 'for', 'old', 'people', 'in', 'rural', 'areas', 'have', 'you', 'got', 'them']]


In [37]:
type(data_words[0])

list

In [40]:
data_words_str = [''] * len(data_words)
for i in range(len(data_words)):
    data_words_str[i] = ' '.join(data_words[i])

In [42]:
# Creating Bigram and Trigram Models



# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words_str, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words_str], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)



/Users/Administrator/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [43]:
# Remove Stopwords, Make Bigrams and Lemmatize

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [35]:
len(data_words_nostops)

641104

In [29]:
# Remove Stop Words        ### This section takes a lot of time
data_words_nostops = remove_stopwords(data_words)

In [44]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_str)

In [46]:
data_words_bigrams

[['t',
  'h',
  'e',
  'r',
  'e',
  ' ',
  'a',
  'r',
  'e',
  ' ',
  't',
  'w',
  'o',
  ' ',
  'n',
  'e',
  'w',
  ' ',
  'o',
  'l',
  'd',
  ' ',
  'a',
  'g',
  'e',
  ' ',
  'i',
  'n',
  's',
  'u',
  'r',
  'a',
  'n',
  'c',
  'e',
  ' ',
  'b',
  'e',
  'n',
  'e',
  'f',
  'i',
  't',
  's',
  ' ',
  'f',
  'o',
  'r',
  ' ',
  'o',
  'l',
  'd',
  ' ',
  'p',
  'e',
  'o',
  'p',
  'l',
  'e',
  ' ',
  'i',
  'n',
  ' ',
  'r',
  'u',
  'r',
  'a',
  'l',
  ' ',
  'a',
  'r',
  'e',
  'a',
  's',
  ' ',
  'h',
  'a',
  'v',
  'e',
  ' ',
  'y',
  'o',
  'u',
  ' ',
  'g',
  'o',
  't',
  ' ',
  't',
  'h',
  'e',
  'm'],
 ['i',
  'f',
  ' ',
  'y',
  'o',
  'u',
  ' ',
  'd',
  'o',
  ' ',
  'n',
  'o',
  't',
  ' ',
  'c',
  'o',
  'm',
  'e',
  ' ',
  't',
  'o',
  ' ',
  's',
  'h',
  'e',
  'n',
  'z',
  'h',
  'e',
  'n',
  ' ',
  's',
  'o',
  'o',
  'n',
  'e',
  'r',
  ' ',
  'o',
  'r',
  ' ',
  'l',
  'a',
  't',
  'e',
  'r',
  ' ',
  'y',
  'o',
  'u',
  'r'

In [0]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

In [0]:

# Do lemmatization keeping only noun, adj, vb, adv    ### This section takes a lot of time
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



In [0]:
print(data_lemmatized[:1])

[['new', 'old', 'age', 'insurance', 'benefit', 'old', 'people', 'rural', 'area', 'get']]


In [0]:
# Create the Dictionary and Corpus needed for Topic Modeling

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1)]]


In [0]:
#print(corpus[:1])

In [0]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus, num_topics=100, id2word=id2word, passes=2, workers=2, dtype=np.float64)
#lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                          id2word=id2word,
#                                          num_topics=100,
#                                          random_state=100,
#                                          update_every=1,
#                                          chunksize=100,
#                                          passes=2,
#                                          alpha='auto',
#                                          per_word_topics=True)

In [0]:
# Save model to disk.
from gensim.test.utils import datapath
#

In [0]:
temp_file = datapath("model_traindata")

In [0]:
lda_model.save(temp_file)

In [0]:
# to load model
# # Load a potentially pretrained model from disk.
lda_model1 = gensim.models.LdaMulticore.load(temp_file)

In [0]:
type(lda_model)

gensim.models.ldamulticore.LdaMulticore

In [0]:
#type(lda_model1)

In [0]:
#lda_model1.save('mymodel')

In [0]:
#lda_model2 = gensim.models.LdaMulticore.load('mymodel')
#type(lda_model2)

In [0]:
#lda_model2

In [0]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Perplexity:  -8.091469173701784

# Coherence Score:  0.5113790308348516


Perplexity:  -8.11024934242394

Coherence Score:  0.5109993418853778


In [0]:
#corpus1 = corpus[:5] # for testing the output

In [0]:
#type(lda_model[corpus1])

In [0]:
#corpus2 = corpus[:10000]

In [0]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    #sent_topics_df = pd.DataFrame()
    sent_topics_df_temp = pd.DataFrame()
    sent_topics_df1 = pd.DataFrame()
    #sent_topics_df1.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    
    sent_topics_df2 = pd.DataFrame()
    #sent_topics_df2.columns = ['Dominant_Topic2', 'Perc_Contribution2', 'Topic_Keywords2']

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        #print(row)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                #print(wp)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df1 = sent_topics_df1.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    
    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        #print(row)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):    
            if j == 1: # => second most dominant topic
                wp2 = ldamodel.show_topic(topic_num)
                topic_keywords2 = ", ".join([word for word, prop in wp2])
                sent_topics_df2 = sent_topics_df2.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                    
            
    
    sent_topics_df1.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    sent_topics_df2.columns = ['Dominant_Topic2', 'Perc_Contribution2', 'Topic_Keywords2']
    #print(sent_topics_df1)
    #print('\n sent_topics_df2 is:')
    #print(sent_topics_df2)

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df_temp = pd.concat([sent_topics_df1, sent_topics_df2, contents], axis=1, join = 'inner')
    #print(sent_topics_df_temp)
    #sent_topics_df = pd.concat([sent_topics_df_temp, contents], axis=1)
     #print(wp)
    return(sent_topics_df_temp)

In [0]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)

In [0]:


# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic1', 'Perc_Contrib1', 'Keywords1', 'Dominant_Topic2', 'Perc_Contribution2', 'Topic_Keywords2', 'Text']

# Show
df_dominant_topic.head(10)

In [0]:
# to find topic number for each row of data
#doc_model = lda_model[corpus]

In [0]:
#type(doc_model)

In [0]:
#doc_topics = list()

In [0]:
#doc_topics.append( lda_model.get_document_topics(corpus[0], minimum_probability=None, minimum_phi_value=None, per_word_topics=False) )

In [0]:
# show topics
lda_model.show_topics(formatted=False)